Axioms for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.physics.quantum</a>
========

In [ ]:
import proveit
# Prepare this notebook for defining the axioms of a theory:
%axioms_notebook # Keep this at the top following 'import proveit'.
from proveit import Function, Lambda, ExprArray, ExprTuple, Composition
from proveit import c, f, k, m, n, x, y, alpha, B, M, U, V, W, fx, fy
from proveit.core_expr_types import A_1_to_m
from proveit.linear_algebra import (
    LinMap, MatrixSpace, MatrixMult, TensorProd, SU, ScalarProd)
from proveit.logic import Equals, Forall, Iff, InSet, Set
from proveit.numbers import zero, one, two, frac, Integer, Natural, NaturalPos, Complex, Exp
from proveit.numbers import Add, Exp, Mult, sqrt, subtract, greater
from proveit.numbers.number_sets import Interval
from proveit.physics.quantum import (
    QuantumOpMult, Qmap, Qmaps_A_1_to_m, 
    Ket, Input, Output, RegisterKet)
from proveit.physics.quantum.circuit import Gate, MultiQubitGate, CircuitEquiv, Circuit, MultiWire
# from proveit.physics.quantum import Bra, Ket, RegisterBra, Meas, MultiWire, Circuit
from proveit.physics.quantum import ket0, ket1, ket_plus, H, QubitSpace, RegisterSU, I, CONTROL
# from proveit.physics.quantum import I, X, Y, Z, RegisterSU
from proveit.physics.quantum import QubitRegisterSpace

In [ ]:
%begin axioms

In [ ]:
ket_zero_in_qubit_space = InSet(ket0, QubitSpace)

In [ ]:
ket_one_in_qubit_space = InSet(ket1, QubitSpace)

In [ ]:
single_qubit_register_ket = Forall(
        x,
        Equals(RegisterKet(x, one), Ket(x)))

In [ ]:
single_qubit_register_zero = Equals(RegisterKet(zero, one), ket0)

In [ ]:
single_qubit_register_one = Equals(RegisterKet(one, one), ket1)

Quantum operations can "multiply" to form a new quantum operation via the composition of linear maps.  We'll just check that the last operand is a linear map to distinguish it from quantum operation application.

In [ ]:
quantum_op_mult_def = Forall(
    m, Forall((V, W), Forall(
        (A_1_to_m, B), Equals(QuantumOpMult(A_1_to_m, B),
                              Composition(Qmaps_A_1_to_m, 
                                          Qmap(B))),
        condition=InSet(Qmap(B), LinMap(V, W))),
    domain=Natural))

Quantum operations can "multiply" along with a ket vector to apply the operations to the vector.  We'll just check that the last two operands are consistent with a linear map application to distinguish it from the "multiplication" of only quantum operators.  

In [ ]:
quantum_op_app_def = Forall(
    m, Forall((V, W), Forall(
        (A_1_to_m, B), Forall(
            x, Equals(QuantumOpMult(A_1_to_m, B, x),
                      Function(Composition(Qmaps_A_1_to_m,
                                           Qmap(B)),
                               x)),
            domain=V),
        condition=InSet(Qmap(B), LinMap(V, W)))),
    domain=Natural)

Quantum map of c-numbers and matrices are defined via multiplication.

In [ ]:
qmap_of_cnumber = Forall(c, Equals(Qmap(c), Lambda(x, Mult(c, x))),
                         domain=Complex)

Don't regard trivial 1-column matrics (which could be regarded as the same as a column-vector) as quantum operators.

In [ ]:
qmap_of_matrix = Forall(
    (m, n), Forall(M, Equals(Qmap(M), Lambda(x, MatrixMult(M, x))),
                   domain=MatrixSpace(Complex, m, n)),
    domain=NaturalPos, condition=greater(n, one))

In [ ]:
scalar_id_for_ket = Forall(k,
                           Equals(Mult(one, Ket(k)), Ket(k)),
                           domain=Natural)

In [ ]:
expand_register_with_zero_ket = Forall(
        n,
        Forall(k,
               Equals(RegisterKet(k, Add(n, one)),
                      TensorProd(ket0, RegisterKet(k, n))),
               domain=Interval(zero, subtract(Exp(two, n), one))),
        domain=NaturalPos)

In [ ]:
expand_register_with_one_ket = Forall(
        n, 
        Forall(k, Equals(RegisterKet(Add(k, Exp(two, n)), Add(n, one)),
                         TensorProd(ket1, RegisterKet(k, n))),
               domain=Interval(zero, subtract(Exp(two, n), one))),
        domain=NaturalPos)

In [ ]:
ket_plus_def = Equals(ket_plus, frac(Add(ket0, ket1), sqrt(two)))

In [ ]:
hadamard_on_zero = Equals(MatrixMult(H, ket0), ket_plus)

In [ ]:
empty_gate = Equals(Gate(), I.with_styles(representation="explicit")) # base case

In [ ]:
substitution = Forall((f, x, y), CircuitEquiv(fx, fy), conditions=CircuitEquiv(x, y))

And then we have several axioms involving the Circuit class, which class itself still needs updating<br/>(in particular, the Circuit class needs an appropriate substitute for the ExpressionTensor class -- perhaps ExprArray).

In [ ]:
circuit_gate_application = Forall(
    U, 
    Forall((x, y),
           Iff(Circuit(ExprArray(ExprTuple(Input(x), Gate(U), Output(y)))),
               Equals(y, MatrixMult(U, x))),
           domain=QubitSpace),
    domain=SU(two))

In [ ]:
circuit_multi_gate_application = Forall(
    n,
    Forall(U,
           Forall((x, y),
                  Iff(Circuit(ExprArray(ExprTuple(Input(x), MultiWire(n), Gate(U), Output(y)))),
                      Equals(y, MatrixMult(U, x))),
                  domain=QubitRegisterSpace(n)),
           domain=RegisterSU(n)),
    domain=NaturalPos)

In [ ]:
zero_controlled_ngate = Forall(
    n,
    Forall(U,
           Forall(x,
                  Forall(alpha,
                         Circuit(ExprArray(ExprTuple(Input(ScalarProd(alpha, ket0)),
                                   I,
                                   MultiQubitGate(CONTROL, Set(one, two)),
                                   Output(ScalarProd(alpha, ket0))),
                                  ExprTuple(Input(x), MultiWire(n), MultiQubitGate(U, Set(one, two)), Output(x)))),
                         domain=Complex),
                  domain=QubitRegisterSpace(n)),
           domain=SU(Exp(two, n))),
    domain=NaturalPos)

In [ ]:
one_controlled_ngate = Forall(
    n,
    Forall(U,
           Forall(x,
                  Forall(alpha,
                         Circuit(ExprArray(ExprTuple(Input(ScalarProd(alpha, ket1)),
                                   I,
                                   MultiQubitGate(CONTROL, Set(one, two)),
                                   Output(ScalarProd(alpha, ket1))),
                                  ExprTuple(Input(x), MultiWire(n), MultiQubitGate(U, Set(one, two)), Output(MatrixMult(U, x))))),
                         domain=Complex),
                  domain=QubitRegisterSpace(n)),
           domain=SU(Exp(two, n))),
    domain=NaturalPos)

In [ ]:
%end axioms